## Hello, torch
Follows the basic torch demonstration of image classifier.

In [142]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from periodiclogger import PeriodicLogger
import random

logger = PeriodicLogger(100)
batch_size = 32
epochs = 1

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [136]:
data_train = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())
data_test = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

train_dataloader = DataLoader(data_test, batch_size=batch_size)
test_dataloader = DataLoader(data_train, batch_size=batch_size)

In [137]:
class NeuralNetwork(nn.Module):
	def __init__(self):
		super().__init__()
		self.flatten = nn.Flatten()
		self.linear_relu_stack = nn.Sequential(
			nn.Linear(28*28, 512),
			nn.ReLU(),
			nn.Linear(512,512),
			nn.ReLU(),
			nn.Linear(512, 10)
		)

	def forward(self, x):
		x = self.flatten(x)
		logits = self.linear_relu_stack(x)
		return logits

In [138]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [139]:
## training
model.train()

for i in range(epochs):
	for (x,y) in test_dataloader:
		x,y = x.to(device), y.to(device)
		pred = model(x)
		loss = loss_fn(pred, y)
		logger.log(loss)

		# backprop
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

tensor(2.2808, grad_fn=<NllLossBackward0>)
tensor(1.0630, grad_fn=<NllLossBackward0>)
tensor(1.0358, grad_fn=<NllLossBackward0>)
tensor(0.6023, grad_fn=<NllLossBackward0>)
tensor(0.4958, grad_fn=<NllLossBackward0>)
tensor(0.4806, grad_fn=<NllLossBackward0>)
tensor(0.4700, grad_fn=<NllLossBackward0>)
tensor(0.4350, grad_fn=<NllLossBackward0>)
tensor(0.2645, grad_fn=<NllLossBackward0>)
tensor(0.4858, grad_fn=<NllLossBackward0>)
tensor(0.3252, grad_fn=<NllLossBackward0>)
tensor(0.5889, grad_fn=<NllLossBackward0>)
tensor(0.4889, grad_fn=<NllLossBackward0>)
tensor(0.5271, grad_fn=<NllLossBackward0>)
tensor(0.6721, grad_fn=<NllLossBackward0>)
tensor(0.2657, grad_fn=<NllLossBackward0>)
tensor(0.4839, grad_fn=<NllLossBackward0>)
tensor(0.4975, grad_fn=<NllLossBackward0>)
tensor(0.4705, grad_fn=<NllLossBackward0>)


In [140]:
## testing
model.eval()
with torch.no_grad():
	for x,y in test_dataloader:
		x,y = x.to(device), y.to(device)
		pred = model(x)
		logger.log(loss_fn(pred, y).item())

0.3239265978336334
0.2035355269908905
0.47838345170021057
0.5534130930900574
0.4317190647125244
0.44649818539619446
0.6523025035858154
0.6425979137420654
0.29126209020614624
0.5109928250312805
0.20605285465717316
0.4719986915588379
0.2802724242210388
0.3853141963481903
0.43016672134399414
0.6802259087562561
0.4341000020503998
0.30087921023368835
0.1962161809206009


In [144]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
for i in range(5):
    j = random.randint(0,10000)
    x, y = data_test[j]
    with torch.no_grad():
        pred = model(x)
        best_index = pred[0].argmax(0)
        predicted, actual = classes[best_index], classes[y]
        logit = pred[0][best_index]
        prob = (1/(1+torch.exp(-logit)))*100
        print(f'Predicted: {predicted}, Actual: {actual}, Confidence: {prob:.2f}%')

Predicted: Sneaker, Actual: Sneaker, Confidence: 99.98%
Predicted: Coat, Actual: Coat, Confidence: 99.30%
Predicted: Dress, Actual: Dress, Confidence: 98.39%
Predicted: Dress, Actual: Dress, Confidence: 98.39%
Predicted: T-shirt/top, Actual: T-shirt/top, Confidence: 99.73%
